# Libraries

In [2]:
import json
import re

import numpy as np
import pandas as pd

from collections import Counter
from pprint import pprint

# Load Data

In [4]:
directory_path = f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE/PilotStudyToEvaluate_DATA_LABELS_2024-08-12_0908.csv'
df = pd.read_csv(directory_path)
df.head(5)

,Study ID,Date subject signed consent,Date of Admission,Age at Admission,Sex,Ethnicity,Smoking History,Does the patient have any relevant medical history? (check all that apply) (choice=None),Does the patient have any relevant medical history? (check all that apply) (choice=Diabetes Mellitus),Does the patient have any relevant medical history? (check all that apply) (choice=Chronic Kidney Disease),...,Any adverse events?.5,"If yes, please specify.5",Is the subject compliant to medications?.5,List all the medications the subject is taking.5,Complete?.6,Date of discharge,Has patient completed study?,Date of study completion,Comments.1,Complete?.7
0,1,14/6/2022,13/6/2022,89,Male,Indian,Never smoked,Unchecked,Checked,Unchecked,...,NaN,NaN,NaN,NaN,Complete,15/6/2022,Yes,15/6/2022,NaN,Complete
1,2,16/6/2022,15/6/2022,68,Male,Chinese,Smoker,Unchecked,Checked,Checked,...,No,NaN,Yes,Aspirin 100mg OM Nifedipine LA 30mg BD Omepr...,Complete,22/6/2022,Yes,21/6/2022,NaN,Complete
2,3,20/6/2022,18/7/2022,81,Male,Chinese,Ex-smoker,Unchecked,Checked,Checked,...,NaN,NaN,NaN,NaN,Complete,23/6/2022,Yes,23/6/2022,NaN,Complete
3,4,18/6/2022,18/6/2022,55,Male,Malay,Ex-smoker,Unchecked,Checked,Unchecked,...,NaN,NaN,NaN,NaN,Complete,22/6/2022,Yes,22/6/2022,NaN,Complete
4,5,21/6/2022,20/6/2022,37,Female,Malay,Never smoked,Unchecked,Checked,Unchecked,...,No,NaN,Yes,ENGERIX B (Hepatitis B Vaccine) 20mcg/mL Injec...,Complete,26/6/2022,Yes,26/7/2022,NaN,Complete


# Sorting Test, Visit, and Discharge

In [5]:
columns = list(df.columns)

list_pointers = []
list_pointers.append((0, 'start'))

counter_test = 0
counter_visit = 0
counter_discharge = 0

for i in range(len(columns)):
    column = columns[i]
    
    if ('Date and time of test' in column) or ('Date/time of test' in column):
        # if 'test' not in list_pointers[-1][1]:        
        list_pointers.append((i, f'test_{counter_test}'))
        counter_test += 1
        
    if ('Date of visit' in column):
        list_pointers.append((i, f'visit_{counter_visit}'))
        counter_visit += 1
        
    if ('Date of discharge' in column):
        list_pointers.append((i, 'discharge')) 
        counter_discharge += 1
        
list_pointers.append((i, 'end'))
pprint(list_pointers)

[(0, 'start'),
 (21, 'test_0'),
 (23, 'test_1'),
 (25, 'test_2'),
 (29, 'visit_0'),
 (54, 'test_3'),
 (61, 'test_4'),
 (68, 'test_5'),
 (76, 'visit_1'),
 (101, 'test_6'),
 (108, 'test_7'),
 (115, 'test_8'),
 (123, 'visit_2'),
 (148, 'test_9'),
 (155, 'test_10'),
 (162, 'test_11'),
 (170, 'visit_3'),
 (195, 'test_12'),
 (202, 'test_13'),
 (209, 'test_14'),
 (217, 'visit_4'),
 (242, 'test_15'),
 (249, 'test_16'),
 (256, 'test_17'),
 (264, 'visit_5'),
 (289, 'test_18'),
 (296, 'test_19'),
 (303, 'test_20'),
 (310, 'discharge'),
 (314, 'end')]


In [6]:
for i in range(len(list_pointers)-1):
    pointer_start = list_pointers[i][0]
    pointer_end = list_pointers[i+1][0]
    
    title = list_pointers[i][1]
    column_subset = columns[pointer_start:pointer_end]
    
    print('')
    print(title, '-', len(column_subset))
    pprint(column_subset)


start - 21
['Study ID',
 'Date subject signed consent',
 'Date of Admission',
 'Age at Admission',
 'Sex',
 'Ethnicity',
 'Smoking History',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=None)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Diabetes Mellitus)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Chronic Kidney Disease)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Hypertension)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Dyslipidemia)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Ischemic Heart Disease)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Atrial fibrillation)',
 'Does the patient have any relevant medical history? (check all that apply) '
 '(choice=Valvular H

# Preprocessing Data

In [7]:
def remove_suffix_from_dict_keys(data):
    return {re.sub(r'\.\d+$', '', key).strip(): value for key, value in data.items()}

list_subject = df['Study ID'].tolist()
dict_subject = {}

for subject in list_subject:
    
    dict_subject[subject] = {
        'Visit': {}, 
        'Test': {},
        'Information': {}
        }
    
    df_subject = df[df['Study ID'] == subject].reset_index(drop=True)
    
    for i in range(len(list_pointers)-1):
        pointer_start = list_pointers[i][0]
        pointer_end = list_pointers[i+1][0]
        title = list_pointers[i][1]
        column_subset = columns[pointer_start:pointer_end]

        json_load = df_subject[column_subset].to_json(orient='records')
        json_load = json.loads(json_load)[0]
        
        json_load = remove_suffix_from_dict_keys(json_load)
        
        if 'Date/time of test' in json_load.keys():
            if json_load['Date/time of test'] == None:
                continue
            
        if 'Date and time of test' in json_load.keys():
            if json_load['Date and time of test'] == None:
                continue
        
        if 'Date of visit' in json_load.keys():
            if json_load['Date of visit'] == None:
                continue
    
        if 'test' in title:   
           dict_subject[subject]['Test'][title] = json_load
        elif 'visit' in title:
           dict_subject[subject]['Visit'][title] = json_load
        else:
           dict_subject[subject]['Information'][title] = json_load

# Get Subject Information

In [9]:
df_summary = []
for subject in list_subject:
    temp = dict_subject[subject]['Information'].copy()
    temp_start = temp['start'].copy()
    temp_discharge = temp['discharge'].copy()
    
    temp_start.update(temp_discharge)

    df_summary.append(temp_start)
    
df_summary = pd.DataFrame(df_summary)
df_summary.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE/subject_summary.csv', index=False)
df_summary.head(5)

,Study ID,Date subject signed consent,Date of Admission,Age at Admission,Sex,Ethnicity,Smoking History,Does the patient have any relevant medical history? (check all that apply) (choice=None),Does the patient have any relevant medical history? (check all that apply) (choice=Diabetes Mellitus),Does the patient have any relevant medical history? (check all that apply) (choice=Chronic Kidney Disease),...,Does the patient have any relevant medical history? (check all that apply) (choice=Chronic Venous Insufficiency),Does the patient have any relevant medical history? (check all that apply) (choice=Prior Episode of Heart Failure),Does the patient have any relevant medical history? (check all that apply) (choice=Diuretic Use Medication),LVEF,LVEF Date,Was Troponin trending done?,Date of discharge,Has patient completed study?,Date of study completion,Comments
0,1,14/6/2022,13/6/2022,89,Male,Indian,Never smoked,Unchecked,Checked,Unchecked,...,Unchecked,Unchecked,Unchecked,40.0,15/7/2021,No,15/6/2022,Yes,15/6/2022,None
1,2,16/6/2022,15/6/2022,68,Male,Chinese,Smoker,Unchecked,Checked,Checked,...,Unchecked,Unchecked,Unchecked,60.0,17/6/2022,Yes,22/6/2022,Yes,21/6/2022,None
2,3,20/6/2022,18/7/2022,81,Male,Chinese,Ex-smoker,Unchecked,Checked,Checked,...,Unchecked,Unchecked,Unchecked,30.0,20/6/2022,Yes,23/6/2022,Yes,23/6/2022,None
3,4,18/6/2022,18/6/2022,55,Male,Malay,Ex-smoker,Unchecked,Checked,Unchecked,...,Unchecked,Checked,Checked,30.0,15/9/2021,Yes,22/6/2022,Yes,22/6/2022,None
4,5,21/6/2022,20/6/2022,37,Female,Malay,Never smoked,Unchecked,Checked,Unchecked,...,Unchecked,Unchecked,Unchecked,60.0,21/6/2022,Yes,26/6/2022,Yes,26/7/2022,None


# Get Visits

In [11]:
df_summary = []
for subject in list_subject:
    temp = dict_subject[subject]['Visit'].copy()

    for key, value in temp.items():
        
        dict_info = {
            'Subject ID': subject,
            'Visit ID': key.replace('visit_', '')
            }
        
        dict_info.update(value)        
        df_summary.append(dict_info)
    
df_summary = pd.DataFrame(df_summary)

# Preprocess numeric details
df_summary['Weight'] = df_summary['Weight'].apply(lambda x: str(x).replace('kg', '').strip() if x is not None else x) # Accidentally add kg in the values
df_summary['Weight'] = df_summary['Weight'].apply(lambda x: str(x).replace('..', '.').strip() if x is not None else x) # Accidentally add ..

df_summary['SPO2'] = df_summary['SPO2'].apply(lambda x: str(x).replace('%', '').strip() if x is not None else x)

leg_columns_all = [
    'Left foot length (cm)', 
    'Left foot width (cm)', 
    'Left foot circumference at center (cm)', 
    'Left foot ankle circumference (cm)',
    'Right foot length (cm)',
    'Right foot width (cm)', 
    'Right foot circumference at center (cm)',
    'Right foot ankle circumference (cm)'
    ]

for col in leg_columns_all:
    df_summary[col] = df_summary[col].apply(lambda x: str(x).replace('..', '.').strip() if x is not None else x)
    df_summary[col] = df_summary[col].apply(lambda x: str(x).replace('/', '.').strip() if x is not None else x)

# Preprocess comments
column = 'How is the subject feeling today'
df_summary[column] = df_summary[column].apply(lambda x: str(x).lower() if x is not None else x)

# Save
df_summary = df_summary.fillna('')
df_summary.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE/subject_visit.csv', index=False)
df_summary.head(5)

,Subject ID,Visit ID,Date of visit,How is the subject feeling today,Is the subject feeling breathless?,Did the Subject sleep well?,Was the height taken,Height,Was weight taken?,Weight,...,"If no, why",Left foot length (cm),Left foot width (cm),Left foot circumference at center (cm),Left foot ankle circumference (cm),Right foot length (cm),Right foot width (cm),Right foot circumference at center (cm),Right foot ankle circumference (cm),FBC done?
0,1,0,14/6/2022,good,No,Yes,Yes,168.0,Yes,67.6,...,,24.0,7.0,25.5,21.5,25.0,7.0,25.0,22.0,Yes
1,1,1,15/6/2022,ok,No,No,Yes,168.0,Yes,67.8,...,patient declined,nan,nan,nan,,nan,nan,nan,nan,No
2,2,0,16/6/2022,ok,No,No,No,,No,,...,,25.5,7.0,26.5,23.0,25.0,8.0,25.0,22.5,Yes
3,2,1,17/6/2022,ok,Yes,No,No,,,,...,patient going for procedure today (time constr...,nan,nan,nan,,nan,nan,nan,nan,No
4,2,2,18/6/2022,not done- weekend,,,No,,No,,...,weekend,nan,nan,nan,,nan,nan,nan,nan,No


# Get Tests

In [15]:
df_summary = []
for subject in list_subject:
    temp = dict_subject[subject]['Test'].copy()

    for key, value in temp.items():
        
        # Update all date time to same column
        if 'Date/time of test' in value.keys():
            value['Date and time of test'] = value.pop('Date/time of test')
            
        # Get type of test
        test_type = ''
        if 'Troponin' in value.keys():
            test_type = 'Heart'
        elif 'Sodium' in value.keys():
            test_type = 'Kidney'
        elif 'Hemoglobin' in value.keys():
            test_type = 'Blood'
        elif 'NT-ProBNP' in value.keys():
            test_type = 'Clinical Trials'
    
        dict_info = {
            'Subject ID': subject,
            'Test ID': key.replace('test_', ''),
            'Test Type': test_type}
        
        dict_info.update(value)
        df_summary.append(dict_info)
    
df_summary = pd.DataFrame(df_summary)

df_summary = df_summary.fillna('')
df_summary['Date and time of test'] = pd.to_datetime(df_summary['Date and time of test'])
df_summary = df_summary.groupby('Subject ID').apply(lambda x: x.sort_values(by='Date and time of test')).reset_index(drop=True)

print('Distribution of tests')
pprint(Counter(df_summary['Test Type'].tolist()))

print(f'No. of tests: {len(df_summary)}')

df_summary.to_csv(f'/home/weiyanpeh/Git/SFM_Related/CADENCE/SHAPE/subject_test.csv', index=False)
df_summary.head(5)    

Distribution of tests
Counter({'Kidney': 202, 'Heart': 125, 'Blood': 88, 'Clinical Trials': 55})
No. of tests: 470


,Subject ID,Test ID,Test Type,Date and time of test,WBC,Hemoglobin,HCT,Platelets,Neutrophils,Renal panel done?,...,Any adverse events?,"If yes, please specify",Is the subject compliant to medications?,List all the medications the subject is taking,Complete?,Was the visit performed?,NT-ProBNP done?,Troponin,Comments,Renal Panel done?
0,1,4,Kidney,2022-06-14 07:47:00,,,,,,,...,,,,,,,,,,
1,1,7,Kidney,2022-06-15 09:33:00,,,,,,,...,,,,,,,No,,,
2,1,3,Blood,2022-10-06 09:44:00,3.95,11.5,34.9,182.0,2.31,Yes,...,,,,,,,,,,
3,1,5,Clinical Trials,2022-10-06 09:44:00,,,,,,,...,No,,Yes,Atorvastatin 40mg ON Bisoprolol 2.5mg BD Col...,Complete,Yes,,,,
4,2,0,Heart,2022-06-15 19:55:00,,,,,,,...,,,,,,,,693.5,,
